# **K-Means Clustering: Grouping of data according to the centroids**

In [ ]:
# import all the required libraries
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_blobs
from scipy.linalg import norm

In [ ]:
# constants
RANDOM_SEED = 150
N_SAMPLES = 1000
N_FEATURES = 2
N_CENTERS = 4

In [ ]:
X, y = make_blobs(
    n_features=N_FEATURES,
    n_samples=N_SAMPLES,
    centers=N_CENTERS,
    random_state=RANDOM_SEED,
)

In [ ]:
print(X.shape, y.shape)

In [ ]:
print(X[:5], y[:5])

In [ ]:
# plot the data
def plot_clusters(X, labels, k):
    colors = ["r", "g", "y", "m"]
    plt.figure(figsize=(8, 8))
    for i in range(k):
        i_label = X[labels == i]  # get all points whose label equals i
        plt.scatter(i_label[:, 0], i_label[:, 1], color=colors[i], marker=".")

Example:

Step 1: labels == 0 → [True, False, True, False, True]

Step 2: X[[True, False, True, False, True]] → Returns rows 0, 2, and 4

In [ ]:
plot_clusters(X, y, N_CENTERS)

**_Steps in K-means<br>_**
_1. Initialize centroids randomly.<br>_
_2. Assign datapoints to closest centroid.<br>_
_3. Update centroid positios.<br>_
_4. Repeat 2 & 3 until convergence.<br>_


*Step-1*

In [ ]:
centroid_idxs = np.random.choice(range(N_SAMPLES), size=N_CENTERS, replace=False)
print(f"Indices of center: {centroid_idxs}")
initial_centers = X[centroid_idxs]
print(f"Centers:{initial_centers}")

In [ ]:
print(f"{X[centroid_idxs]}")

In [ ]:
def plot_clusters_with_centers(X, labels, centers, k):
    colors = ["r", "g", "y", "m"]
    plt.figure(figsize=(8, 8))
    for i in range(k):
        cluster_points = X[labels == i]  # get all points whose label equals i
        plt.scatter(cluster_points[:, 0], cluster_points[:, 1], color=colors[i], marker=".", alpha= 0.4)
    plt.scatter(centers[:, 0], centers[:, 1], color="black", marker="x", s=100) 

In [ ]:
plot_clusters_with_centers(X, y, initial_centers, N_CENTERS)

*Step-2*

In [ ]:
distances = np.empty(shape=(N_SAMPLES, N_CENTERS))
for i, centroid in enumerate(initial_centers):
    distance = np.array(norm(X - centroid, axis=1)) # dimension of distance is 1000*1
    distances[:, i] = distance

*X - centroid:<br>Subtracts the current centroid from all data points (via broadcasting)*

    X = np.array([[1,2], [3,4]])
    centroid = np.array([1,2])
    X - centroid → array([[0,0], [2,2]])

*norm(..., axis=1):<br>Computes Euclidean distance for each point to the centroid<br>Formula: √(Δx² + Δy²)*

    norm([[0,0], [2,2]], axis=1) → array([0., 2.828])

*In each iteration of the loop, the code calculates the distances from all data points to one specific centroid, and stores these distances in a single column of the distances matrix.*

In [ ]:
distances[:5]

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [ ]:
X_scaled[:5]

In [ ]:
centroids = X_scaled[centroid_idxs]
centroids

In [ ]:
# Finding the closest centroid for each point
for i, point in enumerate(X_scaled):
    distances = [norm(point - centroid) for centroid in centroids]
    print(distances)
    print(
        np.argmin(distances)
    )  # returns the minimum value's index along the specified axis
    print(centroids[np.argmin(distances), :])
    
    if i==3:
        break

In [ ]:
def assign_cluster(X, centroids):
    n_samples = X.shape[0]  # or simple N_SAMPLES
    k = len(centroids)
    distances = np.empty(shape=(n_samples, k))
    for i, centroid in enumerate(centroids):
        distances[:, i] = np.array(norm(X - centroid, axis=1))
    # closest_centroid = np.argmin(distances, axis=1)
    return np.argmin(distances, axis=1)

In [ ]:
centroid_for_data = assign_cluster(X_scaled, centroids)

*Step 3*

In [ ]:
# update centroids
new_centroids = []
for i in range(N_CENTERS):
    cluster_data = X_scaled[centroid_for_data == i]
    new_centroids.append(np.mean(cluster_data, axis=0))

In [ ]:
centroids, new_centroids

*Step 4*

In [ ]:
class KMeans:
    def __init__(self, k, tolerance, max_iters):
        self.k = k
        self.toleracne = tolerance
        self.max_iters = max_iters
        self.inertia = 0.0
        self.centroids = []

    # Step 1: Init centroids
    def init_centroids(self, X):
        n_samples = X.shape[0]
        centroid_idxs = np.random.choice(
            range(n_samples), size=self.k, replace=False
        )  # gives the indexes of the centroid from the datapoints
        """
        If Centroid_idxs = 567, that means the features of X at the row 567 gives the
        co-ordinate of the point.
        """
        return X[centroid_idxs]  # returns the centroids's co-ordinates

    # Step 2: Assign points to clusters
    def assign_cluster(self, X):
        n_samples = X.shape[0]
        distances = np.empty(shape=(n_samples, self.k))
        for i in range(self.k):
            distances[:, i] = norm(X - self.centroids[i], axis=1)
        return np.argmin(
            distances, axis=1
        )  # labels - array of indexes of nearest centroid

    # Step3: Update Centroids
    def update_centroids(self, X, labels):
        new_centroids = []
        for i in range(self.k):
            cluster_data = X[labels == i]
            new_centroids.append(np.mean(cluster_data, axis=0))
        return np.array(new_centroids)

    # Measure K-Means performance
    def compute_inertia(self, X, labels):
        for i in range(self.k):
            cluster_data = X[labels == i]
            within_cluster_distance = np.sum(
                norm(cluster_data - self.centroids[i], axis=1) ** 2
            )
            self.inertia += within_cluster_distance
        return self.inertia

    # Combine all Steps
    def fit(self, X):
        self.centroids = self.init_centroids(X)
        for i in range(self.max_iters):
            cluster_labels = self.assign_cluster(X)
            prev_centroids = self.centroids
            self.centroids = self.update_centroids(X, cluster_labels)

            # Check if Converged
            displacement = 0
            for i in range(self.k):
                displacement += norm(prev_centroids[i] - self.centroids[i])

            if displacement < self.toleracne:
                print(f"Converged in {i+1} iterations\n")
                self.compute_inertia(X, cluster_labels)
                return self.centroids, cluster_labels, self.inertia

        self.compute_inertia(X, cluster_labels)
        return self.centroids, cluster_labels, self.inertia

In [ ]:
kmeans = KMeans(k=N_CENTERS, tolerance=1e-4, max_iters=50)

In [ ]:
final_centroids, final_labels, inertia = kmeans.fit(X_scaled)

In [ ]:
print(final_centroids, inertia)

In [ ]:
plot_clusters_with_centers(X_scaled, final_labels, final_centroids, k = N_CENTERS)

In [ ]:
# Elbow Rule
inertia_score = []
for kk in range(2, 20):
    kmeans_obj = KMeans(k=kk, tolerance=1e-4, max_iters=50)
    run_centroids, run_labels, run_inertia = kmeans_obj.fit(X_scaled)
    inertia_score.append(run_inertia)

In [ ]:
plt.figure(figsize=(8, 8))
plt.scatter(range(2, 20), inertia_score, marker="x", c="r")
plt.plot(range(2, 20), inertia_score)
plt.xlabel("k")
plt.ylabel("inertia or WCSS")

In [ ]:
# Compare with SK-learn's KMeans
from sklearn.cluster import KMeans
sk_kmeans = KMeans(n_clusters=N_CENTERS)
clusters = sk_kmeans.fit_transform(X_scaled)
clusters.shape,sk_kmeans.inertia_     

In [ ]:
centers = sk_kmeans.cluster_centers_ 
plt.figure(figsize=(8,8))
plt.scatter(X_scaled[:, 0], X_scaled[:, 1], s=10, c=sk_kmeans.labels_)
plt.scatter(centers[:, 0], centers[:, 1], c='r', s=20)